Installing libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

reading file

In [2]:
df = pd.read_csv('4May23_orders.csv')
df

,SNo,Portfolio Name,Leg ID,Exchange,Exchange Symbol,Product,Order Type,Order ID,Time,Txn,Qty,Filled Qty,Exchg Time,Avg Price,Status,Limit Price,Order Failed,User ID,User Alias,Remarks,Tag
0,1,NF_CSL_STRANGS,1355,NFO,NIFTY 04MAY23 18150 CE,MIS,MARKET,2.020000e+11,04-May-2023 09.18.11,SELL,150,150,04-May-2023 09.18.11,15.90,COMPLETE,0,False,SIM1,SIMULATED1,NaN,10002264
1,2,NF_CSL_STRANGS,1355,NFO,NIFTY 04MAY23 18150 CE,MIS,MARKET,2.310000e+14,04-May-2023 09.18.11,SELL,150,150,01-Jan-0001 00.00.00,15.65,COMPLETE,0,False,50062865,RKEDEL,NaN,10002265
2,3,NF_CSL_STRANGS,1355,NFO,NIFTY 04MAY23 18150 CE,MIS,MARKET,1.100041e+09,04-May-2023 09.18.11,SELL,150,150,01-Jan-0001 00.00.00,15.53,COMPLETE,0,False,ASDE1950,AD1_IIFL,NaN,10002266
3,4,NF_CSL_STRANGS,1356,NFO,NIFTY 04MAY23 18050 PE,MIS,MARKET,2.310000e+14,04-May-2023 09.18.11,SELL,150,150,01-Jan-0001 00.00.00,17.90,COMPLETE,0,False,50062865,RKEDEL,NaN,10002269
4,5,NF_CSL_STRANGS,1356,NFO,NIFTY 04MAY23 18050 PE,MIS,MARKET,1.100041e+09,04-May-2023 09.18.11,SELL,150,150,01-Jan-0001 00.00.00,17.90,COMPLETE,0,False,ASDE1950,AD1_IIFL,NaN,10002270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,212,BNF_25PCT_2RE,1372,NFO,BANKNIFTY 04MAY23 43300 PE,MIS,MARKET,2.020000e+11,04-May-2023 15.09.22,BUY,75,75,04-May-2023 15.09.22,0.35,COMPLETE,0,False,SIM1,SIMULATED1,NaN,10002689
212,213,BNF_25PCT_2RE,1372,NFO,BANKNIFTY 04MAY23 43300 PE,MIS,MARKET,2.310000e+14,04-May-2023 15.09.22,BUY,75,75,01-Jan-0001 00.00.00,0.35,COMPLETE,0,False,50062865,RKEDEL,NaN,10002690
213,214,NF_25PCT_2RE,1374,NFO,NIFTY 04MAY23 18100 PE,MIS,MARKET,2.310000e+14,04-May-2023 15.09.25,BUY,150,150,01-Jan-0001 00.00.00,0.15,COMPLETE,0,False,50062865,RKEDEL,NaN,10002695
214,215,NF_25PCT_2RE,1374,NFO,NIFTY 04MAY23 18100 PE,MIS,MARKET,1.100299e+09,04-May-2023 15.09.25,BUY,150,150,01-Jan-0001 00.00.00,0.15,COMPLETE,0,False,ASDE1950,AD1_IIFL,NaN,10002696


In [3]:
df.insert(5,'Symbol',df['Exchange Symbol'].str.split().str.get(0))
## importing lot sizes
from packages.underlaying import lot_size
df['Symbol'].value_counts()
df.insert(11,'lot_size',df['Symbol'].apply(lot_size))
df.head(2)
df.insert(13,'no_of_lots',(df['Qty']/df['lot_size']).astype(int))
## importing brokerage
from packages.brokerage import brokerage
df.insert(14,'brokerage_per_lot',df['Symbol'].apply(brokerage))
df.insert(15,'brokerage_cost',df['no_of_lots']*df['brokerage_per_lot'])
## importing additional cost
from packages.additionalCost import additional_cost
df.insert(16,'additional_cost',(df['Symbol'].apply(additional_cost)*df['brokerage_cost']).astype(int))
df.insert(17,'Total_cost',(df['brokerage_cost']+df['additional_cost']))
pd.set_option('Display.max_columns',None)
df
def new_price(row):
    if row['Txn'] =='SELL':
        return (row['Avg Price']*row['Qty']-row['Total_cost'])/row['Qty']
    
    else:
        return (row['Avg Price']*row['Qty']+row['Total_cost'])/row['Qty']
df['New Price']= df.apply(new_price,axis=1)
df
def new_amount(row):
    if row['Txn'] == 'SELL':
        return row['New Price']*row['Qty']
    
    else:
        return -(row['New Price']*row['Qty'])
df['New Amount']=df.apply(new_amount,axis=1)
df
df.columns
df['Portfolio Name'].value_counts()
### extracted main portfolio names
df.insert(2,'P_names',df['Portfolio Name'].str.extract('(CSL_STRANGS|70PCT|CP_2RE|25PCT_2RE)'))
grp = df.groupby(['User Alias','P_names'])[['New Price','New Amount']].sum().reset_index()
grp
grp[grp['User Alias']=='AD1_IIFL']['New Amount'].sum()


18795.000000000004